<a href="https://colab.research.google.com/github/StellaNyam/Semantic-Analysis/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.2 MB/s eta 0:00:00


In [3]:
# app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

# Page configuration
st.set_page_config(
    page_title="Mobile Money Forecasting App",
    page_icon="📱",
    layout="wide"
)

# Load models
@st.cache_resource
def load_models():
    try:
        sarima_accounts = joblib.load('sarima_registered_accounts.pkl')
        sarima_volume = joblib.load('sarima_volume.pkl')
        sarima_value = joblib.load('sarima_value.pkl')
        return sarima_accounts, sarima_volume, sarima_value
    except FileNotFoundError:
        st.error("Model files not found. Please ensure the .pkl files are in the same directory.")
        return None, None, None

# Main app
def main():
    st.title("📱 Mobile Money Forecasting App (Kenya)")
    st.markdown("### Predict future mobile money trends using SARIMA models")

    # Load models
    sarima_accounts, sarima_volume, sarima_value = load_models()

    if sarima_accounts is None:
        st.stop()

    # Sidebar controls
    st.sidebar.header("Forecast Parameters")
    steps = st.sidebar.slider(
        "Select number of months to forecast",
        min_value=1,
        max_value=24,
        value=12,
        help="Choose how many months ahead to predict"
    )

    forecast_start = st.sidebar.date_input(
        "Forecast start date",
        value=datetime(2025, 1, 1)
    )

    # Generate forecast button
    if st.sidebar.button("🔮 Generate Forecast", type="primary"):
        with st.spinner("Generating forecasts..."):
            try:
                # Create future dates
                future_dates = pd.date_range(
                    start=forecast_start,
                    periods=steps,
                    freq='M'
                )

                # Generate forecasts
                forecast_accounts = sarima_accounts.forecast(steps=steps)
                forecast_volume = sarima_volume.forecast(steps=steps)
                forecast_value = sarima_value.forecast(steps=steps)

                # Create forecast dataframe
                forecast_df = pd.DataFrame({
                    "Date": future_dates,
                    "Forecasted Registered Accounts (Million)": forecast_accounts,
                    "Forecasted Volume (Million)": forecast_volume,
                    "Forecasted Value (KSh Billion)": forecast_value
                })

                # Display results in tabs
                tab1, tab2, tab3 = st.tabs(["📊 Forecast Table", "📈 Visualizations", "📋 Summary"])

                with tab1:
                    st.subheader("Forecast Results")
                    st.dataframe(
                        forecast_df.round(2),
                        use_container_width=True,
                        hide_index=True
                    )

                    # Download button
                    csv = forecast_df.to_csv(index=False)
                    st.download_button(
                        label="📥 Download Forecast as CSV",
                        data=csv,
                        file_name=f"mobile_money_forecast_{steps}months.csv",
                        mime="text/csv"
                    )

                with tab2:
                    # Interactive plots
                    fig_accounts = px.line(
                        forecast_df,
                        x='Date',
                        y='Forecasted Registered Accounts (Million)',
                        title='Mobile Money Accounts Forecast',
                        markers=True
                    )
                    st.plotly_chart(fig_accounts, use_container_width=True)

                    fig_volume = px.line(
                        forecast_df,
                        x='Date',
                        y='Forecasted Volume (Million)',
                        title='Transaction Volume Forecast',
                        markers=True,
                        color_discrete_sequence=['orange']
                    )
                    st.plotly_chart(fig_volume, use_container_width=True)

                    fig_value = px.line(
                        forecast_df,
                        x='Date',
                        y='Forecasted Value (KSh Billion)',
                        title='Transaction Value Forecast',
                        markers=True,
                        color_discrete_sequence=['green']
                    )
                    st.plotly_chart(fig_value, use_container_width=True)

                with tab3:
                    col1, col2, col3 = st.columns(3)

                    with col1:
                        st.metric(
                            "Final Month Accounts",
                            f"{forecast_accounts.iloc[-1]:.2f}M",
                            f"{forecast_accounts.iloc[-1] - forecast_accounts.iloc[0]:.2f}M"
                        )

                    with col2:
                        st.metric(
                            "Final Month Volume",
                            f"{forecast_volume.iloc[-1]:.2f}M",
                            f"{forecast_volume.iloc[-1] - forecast_volume.iloc[0]:.2f}M"
                        )

                    with col3:
                        st.metric(
                            "Final Month Value",
                            f"{forecast_value.iloc[-1]:.2f}B KSh",
                            f"{forecast_value.iloc[-1] - forecast_value.iloc[0]:.2f}B KSh"
                        )

                    # Growth rates
                    st.subheader("📈 Projected Growth Rates")
                    accounts_growth = ((forecast_accounts.iloc[-1] / forecast_accounts.iloc[0]) - 1) * 100
                    volume_growth = ((forecast_volume.iloc[-1] / forecast_volume.iloc[0]) - 1) * 100
                    value_growth = ((forecast_value.iloc[-1] / forecast_value.iloc[0]) - 1) * 100

                    st.write(f"• **Registered Accounts Growth**: {accounts_growth:.1f}% over {steps} months")
                    st.write(f"• **Transaction Volume Growth**: {volume_growth:.1f}% over {steps} months")
                    st.write(f"• **Transaction Value Growth**: {value_growth:.1f}% over {steps} months")

            except Exception as e:
                st.error(f"Error generating forecast: {str(e)}")

    # Information section
    with st.expander("ℹ️ About this App"):
        st.markdown("""
        This app uses SARIMA (Seasonal AutoRegressive Integrated Moving Average) models to forecast:
        - **Mobile Money Accounts**: Total registered mobile money accounts in millions
        - **Transaction Volume**: Total agent cash in/out volume in millions
        - **Transaction Value**: Total agent cash in/out value in KSh billions

        The models were trained on historical Kenyan mobile money data and can predict up to 24 months ahead.
        """)

if __name__ == "__main__":
    main()

2025-05-27 11:50:09.372 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 11:50:09.383 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 11:50:09.946 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-27 11:50:09.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 11:50:09.957 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 11:50:09.958 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 11:50:09.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn